In [1]:
# 1️⃣ 패키지 설치
!pip install -q openai pandas faiss-cpu langchain langchain-community tiktoken gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# 2️⃣ OpenAI API 키 설정
import os
from google.colab import userdata

# Colab Secrets에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = userdata.get("OPEN_AI")
print("✅ API 키 설정 완료")

✅ API 키 설정 완료


In [3]:
# 3️⃣ CSV 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving 인천광역시_계양구_일반음식점_20250321.csv to 인천광역시_계양구_일반음식점_20250321.csv


In [4]:
# 4️⃣ 데이터 로드 및 전처리
import pandas as pd
import re

# 파일명 확인
file_path = list(uploaded.keys())[0]
print(f"📁 파일명: {file_path}")

# CSV 로드
df = pd.read_csv(file_path, encoding='cp949')
df['영업자시작일'] = pd.to_datetime(df['영업자시작일'], errors='coerce')

print(f"✅ 총 {len(df)}개 데이터 로드 완료")
print("\n📋 컬럼:", df.columns.tolist())
df.head(3)

📁 파일명: 인천광역시_계양구_일반음식점_20250321.csv
✅ 총 2556개 데이터 로드 완료

📋 컬럼: ['연번', '업종명', '업소명', '업태명', '소재지(도로명)', '소재지(지번)', '영업자시작일', '소재지전화', '데이터기준일']


,연번,업종명,업소명,업태명,소재지(도로명),소재지(지번),영업자시작일,소재지전화,데이터기준일
0,1,일반음식점,명태어장,한식,인천광역시 계양구 장제로875번길 4 (임학동),인천광역시 계양구 임학동 10-18,2005-02-16,032-551-9293,2025-03-21
1,2,일반음식점,무한낙지,한식,"인천광역시 계양구 봉오대로 685 (작전동,1층)",인천광역시 계양구 작전동 862-25 1층,2013-06-27,032-544-8555,2025-03-21
2,3,일반음식점,팡팡샌드위치,한식,인천광역시 계양구 안남로 490 (효성동),인천광역시 계양구 효성동 273-1,2020-01-31,032-547-8827,2025-03-21


In [5]:
# 5️⃣ FAISS 벡터 DB 생성
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

def row_to_text(row):
    """각 행을 텍스트로 변환"""
    return (
        f"업소명: {row['업소명']} / "
        f"업태: {row['업태명']} / "
        f"업종: {row['업종명']} / "
        f"주소: {row['소재지(도로명)']} / "
        f"전화번호: {row['소재지전화']} / "
        f"영업시작일: {row['영업자시작일']}"
    )

# Document 객체 생성
texts = df.apply(row_to_text, axis=1).tolist()
docs = [Document(page_content=t) for t in texts]

# 임베딩 및 FAISS DB 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = FAISS.from_documents(docs, embeddings)

print(f"✅ FAISS 벡터 DB 생성 완료 (문서 수: {len(docs)})")

/tmp/ipython-input-1908740498.py:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


✅ FAISS 벡터 DB 생성 완료 (문서 수: 2556)


In [17]:
# 6️⃣ 검색 함수 (FAISS + 필드 필터링 + 제외 필터)
from datetime import datetime
import re

# ✅ 수정된 카테고리 매핑
category_map = {
    '떡볶이': '분식',
    '빈대떡': '한식',
    '삼겹살': '식육',
    '회식': '한식',
    '피자': '피자',
    '피자집': '피자',
    '치킨': '치킨',  # ✅ 간단하게
    '치킨집': '치킨',
    '닭': '치킨',
    '통닭': '치킨',
    '호프': '호프',
    '커피': '카페',
    '커피숍': '카페',
    '중식': '중식',
    '중국집': '중식',
    '한식': '한식',
    '한식당': '한식',
    '일식': '일식',
    '일식집': '일식',
    '분식': '분식',
    '분식집': '분식',
    '고기': '식육',
    '갈비': '식육',
    '술집': '호프',
    '카페': '카페',
    '베이커리': '제과점',
    '빵집': '제과점',
    '빵': '제과점',
    '회': '횟집',
    '횟집': '횟집',
}

# 🚫 제외할 키워드
EXCLUDE_KEYWORDS = [
    '장례', '장례식장', '예식장', '웨딩', '병원', '의원',
    '약국', '편의점', 'GS25', 'CU', '세븐일레븐', '이마트24',
    '회관', '복지관', '주민센터', '동사무소'
]

# 지역 리스트
region_list = ['임학동', '계산동', '작전동', '계양동', '귤현동']

def extract_filters(text):
    """질문에서 연도, 키워드, 지역 추출"""
    years = re.findall(r'\b(20\d{2})\b', text)
    year = int(years[0]) if years else None

    keywords = []
    for key, value in category_map.items():
        if key in text:
            keywords.append(value)
    keywords = list(set(keywords))

    regions = [r for r in region_list if r in text]

    return year, keywords, regions

def is_excluded(row):
    """제외 대상인지 확인"""
    text = f"{row['업소명']} {row['업태명']} {row['업종명']}"
    return any(keyword in text for keyword in EXCLUDE_KEYWORDS)

def search_restaurants(query, top_k=5):
    """FAISS 검색 + 필드 필터링 + 제외 필터"""
    year, keywords, regions = extract_filters(query)

    # 1️⃣ FAISS 검색
    search_results = db.similarity_search(query, k=100)

    # 2️⃣ 인덱스 추출
    result_indices = []
    for doc in search_results:
        try:
            idx = texts.index(doc.page_content)
            result_indices.append(idx)
        except ValueError:
            continue

    # 3️⃣ DataFrame 필터링
    filtered_df = df.iloc[result_indices].copy()

    # 🚫 4️⃣ 제외 키워드 필터링
    filtered_df = filtered_df[~filtered_df.apply(is_excluded, axis=1)]

    # 5️⃣ 연도 필터링
    if year:
        filtered_df = filtered_df[
            pd.notnull(filtered_df['영업자시작일']) &
            (filtered_df['영업자시작일'].dt.year >= year)
        ]

    # 6️⃣ 지역 필터링
    if regions:
        filtered_df = filtered_df[filtered_df.apply(
            lambda x: any(r in str(x['소재지(도로명)']) or r in str(x['소재지(지번)'])
                         for r in regions),
            axis=1
        )]

    # 7️⃣ 업종/업태 키워드 필터링 - ✅ 부분 매칭
    if keywords:
        filtered_df = filtered_df[filtered_df.apply(
            lambda x: any(
                kw.lower() in str(x['업태명']).lower() or
                kw.lower() in str(x['업종명']).lower() or
                kw.lower() in str(x['업소명']).lower()  # ✅ 업소명도 검색
                for kw in keywords
            ),
            axis=1
        )]

    # 8️⃣ 상위 k개
    filtered_df = filtered_df.head(top_k)

    # 9️⃣ 결과 포맷팅
    if filtered_df.empty:
        return f"❌ 검색 결과가 없습니다.\n\n추출된 필터: 연도={year}, 키워드={keywords}, 지역={regions}"

    results = []
    for i, (_, row) in enumerate(filtered_df.iterrows(), 1):
        results.append(
            f"**{i}. {row['업소명']}**\n"
            f"📍 {row['소재지(도로명)']}\n"
            f"☎️ {row['소재지전화'] if pd.notnull(row['소재지전화']) else 'N/A'}\n"
            f"🍽️ {row['업태명']} | {row['업종명']}\n"
            f"📅 개업일: {row['영업자시작일'].date() if pd.notnull(row['영업자시작일']) else 'N/A'}\n"
        )

    return "\n".join(results)

print("✅ 검색 함수 준비 완료")

✅ 검색 함수 준비 완료


In [ ]:
# 7️⃣ Gradio UI 실행
import gradio as gr

def search_with_debug(query):
    """디버깅 정보 포함 검색"""
    year, keywords, regions = extract_filters(query)
    result = search_restaurants(query)

    # 필터 정보 추가
    filter_info = f"🔍 **검색 조건**\n"
    if year:
        filter_info += f"📅 연도: {year}년 이후\n"
    if keywords:
        filter_info += f"🏷️ 음식 종류: {', '.join(keywords)}\n"
    if regions:
        filter_info += f"📍 지역: {', '.join(regions)}\n"
    filter_info += "\n---\n\n"

    return filter_info + result

iface = gr.Interface(
    fn=search_with_debug,  # ✅ 디버깅 버전 사용
    inputs=[
        gr.Textbox(
            label="질문을 입력하세요",
            placeholder="예: 임학동 한식당 추천해줘",
            lines=2
        )
    ],
    outputs=gr.Textbox(label="검색 결과", lines=25),
    title="🍽️ 계양구 일반음식점 AI 검색",
    description=(
        "**인천 계양구 맛집을 AI가 검색해드립니다!**\n\n"
        "지역, 음식 종류, 연도를 자유롭게 조합해서 질문하세요!"
    ),
    examples=[
        ["임학동 한식당 추천해줘"],
        ["2020년 이후 오픈한 피자집"],
        ["계산동 카페"],
        ["작전동 회식 장소"],
        ["치킨집 전화번호"]
    ],
    theme="soft",
    allow_flagging="never"
)

iface.launch(share=True, debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a0570a1a35fe067908.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
